In [ ]:
!pip install keras-ocr

     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 9.5MB 9.8MB/s 
     |████████████████████████████████| 860kB 26.2MB/s 
     |████████████████████████████████| 133kB 44.4MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.6-cp37-none-any.whl size=42937 sha256=1275c39da8dd5a91d5238838e581864569d98a938fbf53910aa589a98beea041
  Stored in directory: /root/.cache/pip/wheels/79/59/e6/79dd15333b911d51ea68d7e4c325e14f042eacc141201dca44
Successfully built keras-ocr


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import glob
import keras_ocr
import json
import string

In [ ]:
!unzip license_plate_detection_data.zip

Archive:  license_plate_detection_data.zip
   creating: license_plate_detection_data/
   creating: license_plate_detection_data/images/
  inflating: license_plate_detection_data/images/wts-lg-000087.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000088.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000089.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000090.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000091.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000094.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000095.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000096.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000097.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000098.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000099.jpg  
  inflating: license_plate_detection_data/images/wts-lg-000100.jpg  
  inflating: license_plate_detection

In [ ]:
def get_licenseplate_detector_dataset(cache_dir=None):
  """
  Args:
      cache_dir: The directory in which to store the data.
  Returns:
      Lists of (image_path, lines, confidence) tuples. Confidence
      is always 1 for this dataset. We record confidence to allow
      for future support for weakly supervised cases.
  """
  if cache_dir == None:
    raise ValueError('cache_dir is None')

  main_dir = os.path.join(cache_dir, 'license_plate_detection_data')
  training_images_dir = os.path.join(main_dir, 'images')
  training_gt_dir = os.path.join(main_dir, 'annotations')

  dataset = []
  for gt_filepath in glob.glob(os.path.join(training_gt_dir, '*.txt')):
    image_id = os.path.split(gt_filepath)[1].split('.')[0]
    image_path = os.path.join(training_images_dir, image_id + '.jpg')
    lines = []
    with open(gt_filepath, 'r') as f:
      current_line = []
      for row in f.read().split('\n'):
        current_line = []
        row = row.split(' ')
        character = row[-1][1:-1]

        x1, y1, x2, y1, x2, y2, x1, y2 = map(int, row[:8])
        current_line.append((np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]), character))
        lines.append(current_line)

    # Some lines only have illegible characters and if skip_illegible is True,
    # then these lines will be blank.
    lines = [line for line in lines if line]
    dataset.append((image_path, lines, 1))

  return dataset

In [ ]:
def get_licenseplate_image_list(cache_dir=None):
  """
  Args:
      cache_dir: The directory in which to store the data.
  Returns:
      Lists of (image_path) lists.
  """
  if cache_dir == None:
    raise ValueError('cache_dir is None')

  main_dir = os.path.join(cache_dir, 'license_plate_detection_data')
  training_images_dir = os.path.join(main_dir, 'images')

  image_list = []
  for image_filepath in glob.glob(os.path.join(training_images_dir, '*.jpg')):
    image_list.append(image_filepath)

  return image_list

In [ ]:
test_result_save_dir = './test_results/'
os.makedirs(test_result_save_dir, exist_ok=True)

### Detector와 Recognizer 불러오기

In [ ]:
detector = keras_ocr.detection.Detector()
detector.model.load_weights('detector_carplate.h5')

recognizer = keras_ocr.recognition.Recognizer()
recognizer.model.load_weights('recognizer_carplate.h5')

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5


In [ ]:
# default scale = 2
pipeline = keras_ocr.pipeline.Pipeline(detector=detector, recognizer=recognizer, scale=1)

In [ ]:
# Get licenseplate image list
image_list = get_licenseplate_image_list(cache_dir='.')

In [ ]:
image_list

['./license_plate_detection_data/images/wts-lg-000073.jpg',
 './license_plate_detection_data/images/wts-lg-000074.jpg',
 './license_plate_detection_data/images/wts-lg-000036.jpg',
 './license_plate_detection_data/images/wts-lg-000023.jpg',
 './license_plate_detection_data/images/wts-lg-000055.jpg',
 './license_plate_detection_data/images/us5.jpg',
 './license_plate_detection_data/images/car12.jpg',
 './license_plate_detection_data/images/c9368c55-210d-456c-a5ef-c310e60039ec.jpg',
 './license_plate_detection_data/images/wts-lg-000124.jpg',
 './license_plate_detection_data/images/car11.jpg',
 './license_plate_detection_data/images/wts-lg-000029.jpg',
 './license_plate_detection_data/images/5b562a61-34ad-4f00-9164-d34abb7a38e4.jpg',
 './license_plate_detection_data/images/us1.jpg',
 './license_plate_detection_data/images/car3.jpg',
 './license_plate_detection_data/images/us7.jpg',
 './license_plate_detection_data/images/a03ced3f-5a97-4e75-8106-fabfd2b8b76e.jpg',
 './license_plate_detectio

### 정성적 성능평가

In [ ]:
from tqdm import tqdm

# Inference and save result to json
keras_ocr_annotations = {}
for image_path in tqdm(image_list):
  images = [
    keras_ocr.tools.read(image_path)
  ]

  # Each list of predictions in prediction_groups is a list of
  # (word, box) tuples.
  prediction_groups = pipeline.recognize(images)

  fig, ax = plt.subplots(nrows=1, figsize=(20, 20))
  keras_ocr.tools.drawAnnotations(image=images[0], predictions=prediction_groups[0], ax=ax)

  # Draw and save result
  plt.draw()
  plt.savefig(test_result_save_dir + image_path.split('/')[-1])

  keras_ocr_annotations[image_path.split('/')[-1]] = [
    {
      'text': text, 'vertices': box.tolist()
    } for text, box in prediction_groups[0]
  ]

# save prediction json file
with open(os.path.join(test_result_save_dir, f'carplate_annotations_scale_1.json'), 'w') as f:
  f.write(json.dumps(keras_ocr_annotations))

### Prediction 결과 이미지 저장

In [ ]:
!zip -r /content/test_results.zip /content/test_results

  adding: content/test_results/ (stored 0%)
  adding: content/test_results/wts-lg-000073.jpg (deflated 11%)
  adding: content/test_results/wts-lg-000074.jpg (deflated 13%)
  adding: content/test_results/wts-lg-000036.jpg (deflated 12%)
  adding: content/test_results/wts-lg-000023.jpg (deflated 17%)
  adding: content/test_results/wts-lg-000055.jpg (deflated 21%)
  adding: content/test_results/us5.jpg (deflated 7%)
  adding: content/test_results/car12.jpg (deflated 6%)
  adding: content/test_results/c9368c55-210d-456c-a5ef-c310e60039ec.jpg (deflated 8%)
  adding: content/test_results/wts-lg-000124.jpg (deflated 8%)
  adding: content/test_results/car11.jpg (deflated 8%)
  adding: content/test_results/wts-lg-000029.jpg (deflated 10%)
  adding: content/test_results/5b562a61-34ad-4f00-9164-d34abb7a38e4.jpg (deflated 10%)
  adding: content/test_results/us1.jpg (deflated 12%)
  adding: content/test_results/car3.jpg (deflated 6%)
  adding: content/test_results/us7.jpg (deflated 13%)
  adding: c

In [ ]:
from google.colab import files
files.download("/content/test_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### ground truth json 형태로 저장

In [ ]:
# read ground truth and save to json
dataset = get_licenseplate_detector_dataset(cache_dir='.')

ground_truth_annotations = {
    image_path.split('/')[-1]: [
        {
            'vertices': lines[0][0][0].astype('float32').tolist(),
            'text': lines[0][0][1],
            'ignore': False
        }
    ]
    for image_path, lines, _  in dataset
}
with open(os.path.join(test_result_save_dir, f'ground_truth_annotations.json'), 'w') as f:
  f.write(json.dumps(ground_truth_annotations))

### 정량적 성능 평가

In [ ]:
# default : iou_threshold : 0.5, similarity_threshold : 0.5
score_kwargs = {
    'iou_threshold': 0.5,
    'similarity_threshold': 1.0,
    'translator': str.maketrans(string.ascii_uppercase, string.ascii_lowercase, string.punctuation),
    'true': ground_truth_annotations
}

for provider, annotations_file in [
  ('carplate_ocr (scale=1)', 'carplate_annotations_scale_1.json')
]:
  with open(os.path.join(test_result_save_dir, annotations_file), 'r') as f:
    annotations = json.loads(f.read())
  results, (precision, recall) = keras_ocr.evaluation.score(pred=annotations, **score_kwargs)
  print(f'{provider}, Precision: {round(precision, 2)}, Recall: {round(recall, 2)}')

carplate_ocr (scale=1), Precision: 0.53, Recall: 0.65
